<a href="https://colab.research.google.com/github/Srinjana/Steganos/blob/main/video_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stegano

In [2]:
from stegano import lsb
from os.path import isfile, join

In [3]:
# import time, opencv, numpy modules
import cv2
import time  
import numpy as np
import math
import os
import shutil
from subprocess import call, STDOUT


In [4]:
def split_string(s_str, count=10):
    per_c = math.ceil(len(s_str)/count)
    c_cout = 0
    out_str = ''
    split_list = []
    for s in s_str:
        out_str += s
        c_cout += 1
        if c_cout == per_c:
            split_list.append(out_str)
            out_str = ''
            c_cout = 0
    if c_cout != 0:
        split_list.append(out_str)
    return split_list

In [5]:
def frame_extraction(video):
    if not os.path.exists("./tmp"):
        os.makedirs("tmp")

    temp_folder = "./tmp"
    print("[INFO] tmp directory has been successfully created.")

    vidcap = cv2.VideoCapture(video)
    count = 0

    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1


In [6]:
def encode_string(input_string, root="./tmp/"):
    split_string_list = split_string(input_string)
    for i in range(0, len(split_string_list)):
        f_name = "{}{}.png".format(root, i)
        secret_enc = lsb.hide(f_name, split_string_list[i])
        secret_enc.save(f_name)
        print("[INFO] frame {} holds {}".format(f_name, split_string_list[i]))


In [7]:
def decode_string(video):
    frame_extraction(video)
    secret = []
    root = "./tmp/"
    for i in range(len(os.listdir(root))):
        f_name = "{}{}.png".format(root, i)
        secret_dec = lsb.reveal(f_name)
        if secret_dec == None:
            break
        secret.append(secret_dec)

    print(''.join([i for i in secret]))
    clean_tmp()


In [8]:
def clean_tmp(path="./tmp"):
    if os.path.exists(path):
        shutil.rmtree(path)
        print("[INFO] tmp files have been cleaned up")


In [9]:
def main():
    input_string = input("Enter the secret Message :")
    f_name = input("Enter the name of video")
    frame_extraction(f_name)
    call(["ffmpeg", "-i", f_name, "-q:a", "0", "-map", "a",
          "tmp/audio.mp3", "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)

    encode_string(input_string)
    call(["ffmpeg", "-i", "tmp/%d.png", "-vcodec", "png", "tmp/video.mov",
          "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)

    call(["ffmpeg", "-i", "tmp/video.mov", "-i", "tmp/audio.mp3", "-codec",
          "copy", "video.mov", "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)
    clean_tmp()

In [10]:
if __name__ == "__main__":
    while True:
        print("1.Hide a secret message in a video \n 2.Reveal the secret message from the video")
        print("Any other key to exit")
        choice = input()
        if choice == '1':
            main()
        elif choice == '2':
            decode_string(input("Enter the name of video file with extension"))
        else:
            break


1.Hide a secret message in a video 
 2.Reveal the secret message from the video
Any other key to exit
1
Enter the secret Message :hakuna matata
Enter the name of videoseal.mp4
[INFO] tmp directory has been successfully created.
[INFO] frame ./tmp/0.png holds ha
[INFO] frame ./tmp/1.png holds ku
[INFO] frame ./tmp/2.png holds na
[INFO] frame ./tmp/3.png holds  m
[INFO] frame ./tmp/4.png holds at
[INFO] frame ./tmp/5.png holds at
[INFO] frame ./tmp/6.png holds a
[INFO] tmp files have been cleaned up
1.Hide a secret message in a video 
 2.Reveal the secret message from the video
Any other key to exit
2
Enter the name of video file with extensionvideo.mov
[INFO] tmp directory has been successfully created.
hakuna matata
[INFO] tmp files have been cleaned up
1.Hide a secret message in a video 
 2.Reveal the secret message from the video
Any other key to exit
n
